<a href="https://colab.research.google.com/github/HikariJadeEmpire/AudioBook_DATAEngineering/blob/main/S01_DATACollection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# In this file, I will :

- Read data from MySQL.
- Read data from a REST API using the package requests.
- Clean data

For educational purposes, I am creating a credential file.

In [ ]:
%%writefile .env

MYSQL_HOST = '34.136.184.58'
MYSQL_PORT = 3306
MYSQL_USER = 'r2de2'
MYSQL_PASSWORD = 'I_Love_Data_Engineer'
MYSQL_DB = 'r2de2'
MYSQL_CHARSET = 'utf8mb4'

Overwriting .env


In [ ]:
!cat .env


MYSQL_HOST = '34.136.184.58'
MYSQL_PORT = 3306
MYSQL_USER = 'r2de2'
MYSQL_PASSWORD = 'I_Love_Data_Engineer'
MYSQL_DB = 'r2de2'
MYSQL_CHARSET = 'utf8mb4'


In [ ]:
! pwd

/content


In [ ]:
! ls -la

total 219216
drwxr-xr-x  1 root root      4096 Jul 18 07:49 .
drwxr-xr-x  1 root root      4096 Jul 18 07:17 ..
drwxr-xr-x  4 root root      4096 Jul 14 13:31 .config
drwx------  5 root root      4096 Jul 18 07:46 drive
-rw-r--r--  1 root root       154 Jul 18 08:38 .env
drwxr-xr-x  1 root root      4096 Jul 14 13:31 sample_data
drwxr-xr-x 13 1000 1000      4096 May 24  2021 spark-3.1.2-bin-hadoop2.7
-rw-r--r--  1 root root 224445805 May 24  2021 spark-3.1.2-bin-hadoop2.7.tgz



Now, I will read the configuration file.

## Connect to Database

In [ ]:
! pip install pymysql
! pip install python-dotenv

In [ ]:
import pymysql
import os
from dotenv import load_dotenv

load_dotenv()

print(os.getenv('MYSQL_PORT'))

# Connect to the database
connection = pymysql.connect(
                              host = os.getenv('MYSQL_HOST'),
                              port = int(os.getenv('MYSQL_PORT')),
                              user = os.getenv('MYSQL_USER'),
                              password = os.getenv('MYSQL_PASSWORD'),
                              db = os.getenv('MYSQL_DB'),
                              charset = os.getenv('MYSQL_CHARSET'),

                              cursorclass = pymysql.cursors.DictCursor
                              )


3306



Cursors are database objects used to access data, with their usage varying depending on the database design. In this case, they will be used as a reference for PyMySQL documentation: https://pymysql.readthedocs.io/en/latest/user/examples.html.

# Read data from MySQL

In [ ]:
# now I'm connected

sql = "SHOW tables;"

with connection.cursor() as cursor:
  cursor.execute(sql)
  result = cursor.fetchall()

print( result )

[{'Tables_in_r2de2': 'audible_data'}, {'Tables_in_r2de2': 'audible_transaction'}]


## You can also connect to the database using Pandas.

In [ ]:
import pandas as pd

sql = "SELECT * FROM audible_transaction"
df_trans = pd.read_sql(sql, connection)

sql2 = "SELECT * FROM audible_data"
df_data = pd.read_sql(sql2, connection)

<ipython-input-43-e9d04ec4d0fb>:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_trans = pd.read_sql(sql, connection)
<ipython-input-43-e9d04ec4d0fb>:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_data = pd.read_sql(sql2, connection)


## Inspect

In [ ]:
display(df_data)
display(df_trans)

,Book_ID,Book Title,Book Subtitle,Book Author,Book Narrator,Audio Runtime,Audiobook_Type,Categories,Rating,Total No. of Ratings,Price
0,1,Bamboozled by Jesus,How God Tricked Me into the Life of My Dreams,Yvonne Orji,Yvonne Orji,6 hrs and 31 mins,Unabridged Audiobook,Biographies & Memoirs,5,47.0,$29.65
1,2,Sixth Realm Part 1,"A LitRPG Fantasy Series (The Ten Realms, Book 6)",Michael Chatfield,Neil Hellegers,13 hrs and 33 mins,Unabridged Audiobook,Science Fiction & Fantasy,4.5,98.0,$24.95
2,3,Go Tell the Bees That I Am Gone,"Outlander, Book 9",Diana Gabaldon,Davina Porter,27 hrs and 30 mins,Unabridged Audiobook,Science Fiction & Fantasy,None,NaN,$41.99
3,4,How the Word Is Passed,A Reckoning with the History of Slavery Across...,Clint Smith,Clint Smith,10 hrs and 7 mins,Unabridged Audiobook,History,None,NaN,$29.65
4,5,The Devil May Dance,A Novel,Jake Tapper,Rob Shapiro,10 hrs and 12 mins,Unabridged Audiobook,"Mystery, Thriller & Suspense",4.5,108.0,$29.65
...,...,...,...,...,...,...,...,...,...,...,...
2264,2265,Kama Sutra,Become a Master of the Art of Love Making. Tan...,Eva Becker,ChasMandala,3 hrs and 29 mins,Unabridged Audiobook,Erotica,5,203.0,$14.95
2265,2266,Meditación Práctica para Principiantes [Prac...,Una guía para principiantes para aliviar la an...,Taylor Knox,William Ditilio,3 hrs and 14 mins,Unabridged Audiobook,Health & Wellness,5,203.0,$14.95
2266,2267,Forgotten,"The Untold Story of D-day's Black Heroes, at H...",Linda Hervieux,Ron Butler,9 hrs and 33 mins,Unabridged Audiobook,History,4.6,70.0,$24.95
2267,2268,The Missing,None,Daisy Pearce,Karen Cass,10 hrs and 18 mins,Unabridged Audiobook,"Mystery, Thriller & Suspense",4.2,141.0,$30.09


,timestamp,user_id,book_id,country
0,2021-05-01 00:00:01,ad8eca41,1584,Portugal
1,2021-05-01 00:00:03,561b26c1,829,United States of America
2,2021-05-01 00:00:04,81f149e5,1391,Japan
3,2021-05-01 00:00:07,4f218413,1586,Taiwan
4,2021-05-01 00:00:18,a4066781,300,United States of America
...,...,...,...,...
1998816,2021-07-31 23:59:47,72da1411,600,Portugal
1998817,2021-07-31 23:59:48,620c27c8,462,Montenegro
1998818,2021-07-31 23:59:55,84fe88ab,1656,United States of America
1998819,2021-07-31 23:59:57,fd6cc4fc,522,United States of America


# Get DATA from Rest API
Guide : https://realpython.com/python-requests/

In [ ]:
# In general, we need to install this package ourselves, but for Colab, it comes pre-installed.
# install : pip install requests

import requests

In [ ]:
url = "https://r2de2-workshop-vmftiryt6q-ts.a.run.app/usd_thb_conversion_rate"

x = requests.get(url)

if x.status_code == 200: # or >>> if x :
    print('Success!')
elif x.status_code == 404: # or >>> else :
    print('Not Found.')

Success!


In [ ]:
print(type(x.json()))
assert isinstance(x.json(), dict)

<class 'dict'>


In [ ]:
df = pd.DataFrame.from_dict(x.json())
df

,conversion_rate
2021-04-01,31.194
2021-04-02,31.290
2021-04-03,31.256
2021-04-04,31.244
2021-04-05,31.342
...,...
2021-08-08,33.395
2021-08-09,33.464
2021-08-10,33.460
2021-08-11,33.145


# now we clean the DATA

In [ ]:
df = df.reset_index().rename(columns={"index": "date"})
df

,date,conversion_rate
0,2021-04-01,31.194
1,2021-04-02,31.290
2,2021-04-03,31.256
3,2021-04-04,31.244
4,2021-04-05,31.342
...,...,...
129,2021-08-08,33.395
130,2021-08-09,33.464
131,2021-08-10,33.460
132,2021-08-11,33.145


In [ ]:
trans = df_trans.merge(df_data, how="left", left_on="book_id", right_on="Book_ID")
trans

,timestamp,user_id,book_id,country,Book_ID,Book Title,Book Subtitle,Book Author,Book Narrator,Audio Runtime,Audiobook_Type,Categories,Rating,Total No. of Ratings,Price
0,2021-05-01 00:00:01,ad8eca41,1584,Portugal,1584,Sycamore Row,None,John Grisham,Michael Beck,20 hrs and 46 mins,Unabridged Audiobook,"Mystery, Thriller & Suspense",4.6,25863.0,$35.00
1,2021-05-01 00:00:03,561b26c1,829,United States of America,829,The 5 Love Languages of Children,The Secret to Loving Children Effectively,Gary Chapman,Chris Fabry,5 hrs and 48 mins,Unabridged Audiobook,"Relationships, Parenting & Personal Development",4.8,2441.0,$19.59
2,2021-05-01 00:00:04,81f149e5,1391,Japan,1391,The Guardians,A Novel,John Grisham,Michael Beck,11 hrs and 50 mins,Unabridged Audiobook,"Mystery, Thriller & Suspense",4.6,26781.0,$31.50
3,2021-05-01 00:00:07,4f218413,1586,Taiwan,1586,How to Be a 3% Man,None,Corey Wayne,Corey Wayne,7 hrs and 33 mins,Unabridged Audiobook,"Relationships, Parenting & Personal Development",4.8,11459.0,$19.95
4,2021-05-01 00:00:18,a4066781,300,United States of America,300,Napoleon Hill's Outwitting the Devil,The Secret to Freedom and Success,Napoleon Hill,Dan John Miller,5 hrs and 51 mins,Unabridged Audiobook,Business & Careers,4.8,9305.0,$17.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1998816,2021-07-31 23:59:47,72da1411,600,Portugal,600,The Time of Contempt,"The Witcher, Book 2",Andrzej Sapkowski,Peter Kenny,11 hrs and 55 mins,Unabridged Audiobook,Science Fiction & Fantasy,4.7,14344.0,$25.08
1998817,2021-07-31 23:59:48,620c27c8,462,Montenegro,462,Endurance,Shackleton's Incredible Voyage,Alfred Lansing,Simon Prebble,10 hrs and 20 mins,Unabridged Audiobook,History,4.7,18764.0,$24.95
1998818,2021-07-31 23:59:55,84fe88ab,1656,United States of America,1656,The Spy and the Traitor,The Greatest Espionage Story of the Cold War,Ben Macintyre,John Lee,13 hrs and 20 mins,Unabridged Audiobook,Biographies & Memoirs,4.8,7150.0,$31.95
1998819,2021-07-31 23:59:57,fd6cc4fc,522,United States of America,522,Lilac Girls,A Novel,Martha Hall Kelly,Cassandra Campbell,17 hrs and 30 mins,Unabridged Audiobook,Literature & Fiction,4.7,19401.0,$31.50


In [ ]:
display(df.info())
print('\n\n')
display(trans.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             134 non-null    object 
 1   conversion_rate  134 non-null    float64
dtypes: float64(1), object(1)
memory usage: 2.2+ KB


None




<class 'pandas.core.frame.DataFrame'>
Int64Index: 1998821 entries, 0 to 1998820
Data columns (total 15 columns):
 #   Column                Dtype         
---  ------                -----         
 0   timestamp             datetime64[ns]
 1   user_id               object        
 2   book_id               int64         
 3   country               object        
 4   Book_ID               int64         
 5   Book Title            object        
 6   Book Subtitle         object        
 7   Book Author           object        
 8   Book Narrator         object        
 9   Audio Runtime         object        
 10  Audiobook_Type        object        
 11  Categories            object        
 12  Rating                object        
 13  Total No. of Ratings  float64       
 14  Price                 object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(11)
memory usage: 244.0+ MB


None

In [ ]:
trans['date'] = trans['timestamp']

df['date'] = pd.to_datetime(df['date']).dt.date
trans['date'] = pd.to_datetime(trans['date']).dt.date

display( df.dtypes )
print( '\n' )
display( trans.dtypes )

date                object
conversion_rate    float64
dtype: object

timestamp               datetime64[ns]
user_id                         object
book_id                          int64
country                         object
Book_ID                          int64
Book Title                      object
Book Subtitle                   object
Book Author                     object
Book Narrator                   object
Audio Runtime                   object
Audiobook_Type                  object
Categories                      object
Rating                          object
Total No. of Ratings           float64
Price                           object
date                            object
dtype: object

In [ ]:
final_df = trans.merge(df, how="left", left_on="date", right_on="date").drop(columns='date')
final_df

,timestamp,user_id,book_id,country,Book_ID,Book Title,Book Subtitle,Book Author,Book Narrator,Audio Runtime,Audiobook_Type,Categories,Rating,Total No. of Ratings,Price,conversion_rate
0,2021-05-01 00:00:01,ad8eca41,1584,Portugal,1584,Sycamore Row,None,John Grisham,Michael Beck,20 hrs and 46 mins,Unabridged Audiobook,"Mystery, Thriller & Suspense",4.6,25863.0,$35.00,31.140
1,2021-05-01 00:00:03,561b26c1,829,United States of America,829,The 5 Love Languages of Children,The Secret to Loving Children Effectively,Gary Chapman,Chris Fabry,5 hrs and 48 mins,Unabridged Audiobook,"Relationships, Parenting & Personal Development",4.8,2441.0,$19.59,31.140
2,2021-05-01 00:00:04,81f149e5,1391,Japan,1391,The Guardians,A Novel,John Grisham,Michael Beck,11 hrs and 50 mins,Unabridged Audiobook,"Mystery, Thriller & Suspense",4.6,26781.0,$31.50,31.140
3,2021-05-01 00:00:07,4f218413,1586,Taiwan,1586,How to Be a 3% Man,None,Corey Wayne,Corey Wayne,7 hrs and 33 mins,Unabridged Audiobook,"Relationships, Parenting & Personal Development",4.8,11459.0,$19.95,31.140
4,2021-05-01 00:00:18,a4066781,300,United States of America,300,Napoleon Hill's Outwitting the Devil,The Secret to Freedom and Success,Napoleon Hill,Dan John Miller,5 hrs and 51 mins,Unabridged Audiobook,Business & Careers,4.8,9305.0,$17.49,31.140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1998816,2021-07-31 23:59:47,72da1411,600,Portugal,600,The Time of Contempt,"The Witcher, Book 2",Andrzej Sapkowski,Peter Kenny,11 hrs and 55 mins,Unabridged Audiobook,Science Fiction & Fantasy,4.7,14344.0,$25.08,32.887
1998817,2021-07-31 23:59:48,620c27c8,462,Montenegro,462,Endurance,Shackleton's Incredible Voyage,Alfred Lansing,Simon Prebble,10 hrs and 20 mins,Unabridged Audiobook,History,4.7,18764.0,$24.95,32.887
1998818,2021-07-31 23:59:55,84fe88ab,1656,United States of America,1656,The Spy and the Traitor,The Greatest Espionage Story of the Cold War,Ben Macintyre,John Lee,13 hrs and 20 mins,Unabridged Audiobook,Biographies & Memoirs,4.8,7150.0,$31.95,32.887
1998819,2021-07-31 23:59:57,fd6cc4fc,522,United States of America,522,Lilac Girls,A Novel,Martha Hall Kelly,Cassandra Campbell,17 hrs and 30 mins,Unabridged Audiobook,Literature & Fiction,4.7,19401.0,$31.50,32.887


In [ ]:
final_df["Price"] = final_df["Price"].apply(lambda x: x.replace("$",""))
final_df["Price"] = final_df["Price"].astype(float)

final_df["Price"]

0          35.00
1          19.59
2          31.50
3          19.95
4          17.49
           ...  
1998816    25.08
1998817    24.95
1998818    31.95
1998819    31.50
1998820    20.52
Name: Price, Length: 1998821, dtype: float64

In [ ]:
final_df["THB_Price"] = final_df["Price"]*final_df["conversion_rate"]
final_df.head()

,timestamp,user_id,book_id,country,Book_ID,Book Title,Book Subtitle,Book Author,Book Narrator,Audio Runtime,Audiobook_Type,Categories,Rating,Total No. of Ratings,Price,conversion_rate,THB_Price
0,2021-05-01 00:00:01,ad8eca41,1584,Portugal,1584,Sycamore Row,None,John Grisham,Michael Beck,20 hrs and 46 mins,Unabridged Audiobook,"Mystery, Thriller & Suspense",4.6,25863.0,35.00,31.14,1089.9000
1,2021-05-01 00:00:03,561b26c1,829,United States of America,829,The 5 Love Languages of Children,The Secret to Loving Children Effectively,Gary Chapman,Chris Fabry,5 hrs and 48 mins,Unabridged Audiobook,"Relationships, Parenting & Personal Development",4.8,2441.0,19.59,31.14,610.0326
2,2021-05-01 00:00:04,81f149e5,1391,Japan,1391,The Guardians,A Novel,John Grisham,Michael Beck,11 hrs and 50 mins,Unabridged Audiobook,"Mystery, Thriller & Suspense",4.6,26781.0,31.50,31.14,980.9100
3,2021-05-01 00:00:07,4f218413,1586,Taiwan,1586,How to Be a 3% Man,None,Corey Wayne,Corey Wayne,7 hrs and 33 mins,Unabridged Audiobook,"Relationships, Parenting & Personal Development",4.8,11459.0,19.95,31.14,621.2430
4,2021-05-01 00:00:18,a4066781,300,United States of America,300,Napoleon Hill's Outwitting the Devil,The Secret to Freedom and Success,Napoleon Hill,Dan John Miller,5 hrs and 51 mins,Unabridged Audiobook,Business & Careers,4.8,9305.0,17.49,31.14,544.6386


## Export

In [ ]:
final_df.to_csv("/content/drive/MyDrive/myColab_Notebooks/myData.csv",index=False)